## Workflow

(for v0.3.0)
https://github.com/ASFHyP3/hyp3-isce2/releases/tag/v0.3.0

https://github.com/ASFHyP3/hyp3-isce2/tree/v0.3.0


1. Find a burst to process
2. Search ASF for scenes
3. Select dates to process
4. Create processing subfolders to execute burst processing in

In [1]:
import geopandas as gpd
import asf_search as asf
import requests
import os
# not sure why DEBUG statements is happening in other libraries if imported after hyp3_isce2...
import logging
import hyp3_isce2.burst as hb # https://github.com/ASFHyP3/hyp3-isce2/issues/53 ?
rootlogs = logging.getLogger()
rootlogs.setLevel('WARNING')
import numpy as np
import xmlschema
import lxml
import re
import shapely

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f25cc69fb50, execution_count=1 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f25cc66bcd0, raw_cell="import geopandas as gpd
import asf_search as asf
i.." store_history=True silent=False shell_futures=True cell_id=ca8e5b53-a8b3-43f5-8f51-c842a7046d16> result=None>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [2]:
import hyp3_isce2
hyp3_isce2.__version__ # syntax changing rapidly, so stick with 0.3.0

'0.9.2.dev0+g6216fc0.d20231127'

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f25571c4150, execution_count=2 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f25579f1c10, raw_cell="import hyp3_isce2
hyp3_isce2.__version__ # syntax .." store_history=True silent=False shell_futures=True cell_id=f23cb40a-ade2-4d07-9297-ea7b4369883b> result='0.9.2.dev0+g6216fc0.d20231127'>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [3]:
# https://geojson.io/#new&map=15.23/47.654452/-122.303174
gfa = gpd.GeoDataFrame.from_features( {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          120.29098,
          37.65524
        ],
        "type": "Point"
      }
    }
  ]
},
 crs=4326   
)
gfa.explore()

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f25c8928690, execution_count=3 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f25574029d0, raw_cell="# https://geojson.io/#new&map=15.23/47.654452/-122.." store_history=True silent=False shell_futures=True cell_id=2545af69-c475-4d85-82ca-b9fbf2739557> result=<folium.folium.Map object at 0x7f2551cd8c10>>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

## Find a single burst

1. You need to download the burst database

https://sar-mpc.eu/test-data-sets/

Sentinel-1 Burst ID Map, version 20220530, generated by the SAR-MPC service, available on Sentinel-1 MPC Test data sets

In [4]:
# For now consider selecting a burst covering a point
gf = gpd.read_file('burst_map_IW_000001_375887.sqlite3',
                   mask=gfa)

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f2552f76e50, execution_count=4 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f25531ab3d0, raw_cell="# For now consider selecting a burst covering a po.." store_history=True silent=False shell_futures=True cell_id=c0042ab6-b500-4a47-8d4a-0f36ea8021bb> result=None>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [5]:
gf.head()

,burst_id,subswath_name,relative_orbit_number,time_from_anx_sec,orbit_pass,geometry
0,5148,IW3,3,2351.899266,DESCENDING,"MULTIPOLYGON Z (((121.12510 37.62819 0.00000, ..."
1,161946,IW1,76,2347.956633,DESCENDING,"MULTIPOLYGON Z (((121.03760 37.59191 0.00000, ..."
2,365368,IW2,171,607.902997,ASCENDING,"MULTIPOLYGON Z (((119.91344 37.49934 0.00000, ..."


Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f25579f16d0, execution_count=5 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f2551604050, raw_cell="gf.head()" store_history=True silent=False shell_futures=True cell_id=a148e1b1-389d-4806-a8fc-5cee18294596> result=   burst_id subswath_name  relative_orbit_number  time_from_anx_sec  \
0      5148           IW3                      3        2351.899266   
1    161946           IW1                     76        2347.956633   
2    365368           IW2                    171         607.902997   

   orbit_pass                                           geometry  
0  DESCENDING  MULTIPOLYGON Z (((121.12510 37.62819 0.00000, ...  
1  DESCENDING  MULTIPOLYGON Z (((121.03760 37.59191 0.00000, ...  
2   ASCENDING  MULTIPOLYGON Z (((119.91344 37.49934 0.00000, ...  >,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [6]:
gf.explore()

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f2551613490, execution_count=6 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f2552f29990, raw_cell="gf.explore()" store_history=True silent=False shell_futures=True cell_id=422131cd-a932-49a7-b8e0-e074ac8fa62e> result=<folium.folium.Map object at 0x7f25515e7590>>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [7]:
# Ascending test
burstId = '171_IW2_365368'

relorb = int(burstId.split('_')[0])
subswath = burstId.split('_')[1]
idnum = int(burstId.split('_')[2])

ind = (gf.relative_orbit_number == relorb) & (gf.subswath_name == subswath) & (gf.burst_id == idnum)
print(relorb)
myburst = gf[ind]
#myburst
print(myburst.geometry.values[0].centroid)
print(myburst.geometry.values[0])


171
POINT (120.40245527692386 37.66329296094743)
MULTIPOLYGON Z (((119.913438 37.499336 0, 120.434347 37.571215 0, 120.926689 37.636799 0, 120.884975 37.82922 0, 120.39225 37.759472 0, 119.870939 37.683421 0, 119.913438 37.499336 0)))
Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f2551617490, execution_count=7 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f25515e72d0, raw_cell="# Ascending test
burstId = '171_IW2_365368'

relor.." store_history=True silent=False shell_futures=True cell_id=70426c89-b873-47fd-a35e-0dd214927a07> result=None>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

## Search ASF Archive

In [8]:
results = asf.geo_search(platform=[asf.PLATFORM.SENTINEL1],
                         processingLevel=asf.SLC,
                         beamMode=asf.BEAMMODE.IW,
                         relativeOrbit=relorb,
                         intersectsWith=str(myburst.geometry.values[0].centroid),
                         start='2000-1-1T00:00:00UTC',
                         end='2099-1-1T00:00:00UTC'
                        )
#print(results)
gf = gpd.GeoDataFrame.from_features(results.geojson(), crs=4326)
for i in range(len(gf)):
    #print(gf.geometry.values[i].intersection(myburst.geometry.values[0]).area)
    print(gf.fileID.values[i])

S1A_IW_SLC__1SDV_20240110T095716_20240110T095743_052043_064A1D_116A-SLC
S1A_IW_SLC__1SDV_20240110T095650_20240110T095718_052043_064A1D_75E7-SLC
S1A_IW_SLC__1SDV_20231229T095651_20231229T095728_051868_064426_76AB-SLC
S1A_IW_SLC__1SDV_20231217T095651_20231217T095728_051693_063E16_4E22-SLC
S1A_IW_SLC__1SDV_20231205T095652_20231205T095729_051518_0637F8_E127-SLC
S1A_IW_SLC__1SDV_20231123T095718_20231123T095745_051343_0631F6_17E5-SLC
S1A_IW_SLC__1SDV_20231123T095652_20231123T095720_051343_0631F6_B247-SLC
S1A_IW_SLC__1SDV_20231111T095718_20231111T095745_051168_062BE8_CE44-SLC
S1A_IW_SLC__1SDV_20231111T095653_20231111T095720_051168_062BE8_40DA-SLC
S1A_IW_SLC__1SDV_20231030T095653_20231030T095730_050993_0625E6_8437-SLC
S1A_IW_SLC__1SDV_20231018T095719_20231018T095746_050818_061FF3_1888-SLC
S1A_IW_SLC__1SDV_20231018T095653_20231018T095721_050818_061FF3_C5A6-SLC
S1A_IW_SLC__1SDV_20231006T095719_20231006T095746_050643_0619EC_0EB4-SLC
S1A_IW_SLC__1SDV_20231006T095653_20231006T095721_050643_0619EC_C

TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [9]:
# 1st acquisition of every month
#gf.groupby([gf.index.year, gf.index.month]).first()

# 1st acquisition of given month every year
subset = gf#[gf.index.month == 1] 
subset = subset#.groupby(subset.index.year).first()

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f25513dd750, execution_count=9 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f25621b8110, raw_cell="# 1st acquisition of every month
#gf.groupby([gf.i.." store_history=True silent=False shell_futures=True cell_id=f9826a50-82b1-4242-b0d9-0b7d4ed4e6a5> result=None>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [10]:
subset

,geometry,beamModeType,browse,bytes,centerLat,centerLon,faradayRotation,fileID,flightDirection,groupID,...,processingDate,processingLevel,sceneName,sensor,startTime,stopTime,url,pgeVersion,fileName,frameNumber
0,"POLYGON ((118.55290 39.01779, 118.95319 37.397...",IW,None,7665833160,38.4153,120.1865,None,S1A_IW_SLC__1SDV_20240110T095716_20240110T0957...,ASCENDING,S1A_IWDV_0120_0127_052043_171,...,2024-01-10T09:57:16.146Z,SLC,S1A_IW_SLC__1SDV_20240110T095716_20240110T0957...,C-SAR,2024-01-10T09:57:16.146Z,2024-01-10T09:57:43.102Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.71,S1A_IW_SLC__1SDV_20240110T095716_20240110T0957...,121
1,"POLYGON ((118.92621 37.52652, 119.33179 35.847...",IW,None,7971013535,36.8947,120.5317,None,S1A_IW_SLC__1SDV_20240110T095650_20240110T0957...,ASCENDING,S1A_IWDV_0115_0122_052043_171,...,2024-01-10T09:56:50.369Z,SLC,S1A_IW_SLC__1SDV_20240110T095650_20240110T0957...,C-SAR,2024-01-10T09:56:50.369Z,2024-01-10T09:57:18.284Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.71,S1A_IW_SLC__1SDV_20240110T095650_20240110T0957...,116
2,"POLYGON ((118.78477 38.08155, 119.32674 35.847...",IW,None,10705033744,37.1721,120.4627,None,S1A_IW_SLC__1SDV_20231229T095651_20231229T0957...,ASCENDING,S1A_IWDV_0115_0123_051868_171,...,2023-12-29T09:56:51.007Z,SLC,S1A_IW_SLC__1SDV_20231229T095651_20231229T0957...,C-SAR,2023-12-29T09:56:51.007Z,2023-12-29T09:57:28.153Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.71,S1A_IW_SLC__1SDV_20231229T095651_20231229T0957...,116
3,"POLYGON ((118.78420 38.08128, 119.32612 35.847...",IW,None,10704952926,37.1720,120.4619,None,S1A_IW_SLC__1SDV_20231217T095651_20231217T0957...,ASCENDING,S1A_IWDV_0115_0123_051693_171,...,2023-12-17T09:56:51.813Z,SLC,S1A_IW_SLC__1SDV_20231217T095651_20231217T0957...,C-SAR,2023-12-17T09:56:51.813Z,2023-12-17T09:57:28.952Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.71,S1A_IW_SLC__1SDV_20231217T095651_20231217T0957...,116
4,"POLYGON ((118.78315 38.08145, 119.32542 35.846...",IW,None,10705000738,37.1716,120.4612,None,S1A_IW_SLC__1SDV_20231205T095652_20231205T0957...,ASCENDING,S1A_IWDV_0115_0123_051518_171,...,2023-12-05T09:56:52.420Z,SLC,S1A_IW_SLC__1SDV_20231205T095652_20231205T0957...,C-SAR,2023-12-05T09:56:52.420Z,2023-12-05T09:57:29.580Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.71,S1A_IW_SLC__1SDV_20231205T095652_20231205T0957...,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,"POLYGON ((118.55096 39.01766, 118.95129 37.397...",IW,None,3656798439,38.4151,120.1845,None,S1A_IW_SLC__1SDV_20170416T095634_20170416T0957...,ASCENDING,S1A_IWDV_0120_0127_016168_171,...,2017-04-16T09:56:34.000Z,SLC,S1A_IW_SLC__1SDV_20170416T095634_20170416T0957...,C-SAR,2017-04-16T09:56:34.000Z,2017-04-16T09:57:01.000Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,002.82,S1A_IW_SLC__1SDV_20170416T095634_20170416T0957...,120
441,"POLYGON ((118.92149 37.52571, 119.32706 35.847...",IW,None,4454664321,36.8943,120.5275,None,S1A_IW_SLC__1SDV_20170416T095609_20170416T0956...,ASCENDING,S1A_IWDV_0116_0121_016168_171,...,2017-04-16T09:56:09.000Z,SLC,S1A_IW_SLC__1SDV_20170416T095609_20170416T0956...,C-SAR,2017-04-16T09:56:09.000Z,2017-04-16T09:56:36.000Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,002.82,S1A_IW_SLC__1SDV_20170416T095609_20170416T0956...,116
442,"POLYGON ((118.55045 39.01878, 118.95091 37.397...",IW,None,3690529770,38.4161,120.1842,None,S1A_IW_SLC__1SDV_20170404T095634_20170404T0957...,ASCENDING,S1A_IWDV_0121_0126_015993_171,...,2017-04-04T09:56:34.000Z,SLC,S1A_IW_SLC__1SDV_20170404T095634_20170404T0957...,C-SAR,2017-04-04T09:56:34.000Z,2017-04-04T09:57:01.000Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,002.82,S1A_IW_SLC__1SDV_20170404T095634_20170404T0957...,121
443,"POLYGON ((118.92114 37.52622, 119.32668 35.847...",IW,None,4445238338,36.8949,120.5272,None,S1A_IW_SLC__1SDV_20170404T095608_20170404T0956...,ASCENDING,S1A_IWDV_0116_0121_015993_171,...,2017-04-04T09:56:08.00

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f2551647290, execution_count=10 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f25501c3110, raw_cell="subset" store_history=True silent=False shell_futures=True cell_id=8d312d73-d9ff-43f1-b0df-9ab2015ea0a0> result=                                              geometry beamModeType browse  \
0    POLYGON ((118.55290 39.01779, 118.95319 37.397...           IW   None   
1    POLYGON ((118.92621 37.52652, 119.33179 35.847...           IW   None   
2    POLYGON ((118.78477 38.08155, 119.32674 35.847...           IW   None   
3    POLYGON ((118.78420 38.08128, 119.32612 35.847...           IW   None   
4    POLYGON ((118.78315 38.08145, 119.32542 35.846...           IW   None   
..                                                 ...          ...    ...   
440  POLYGON ((118.55096 39.01766, 118.95129 37.39

TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [11]:
subset.reset_index(drop=True).set_crs(4326).explore(column='startTime', style_kwds=dict(fillOpacity=.3))

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f2551603b50, execution_count=11 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f2551610e10, raw_cell="subset.reset_index(drop=True).set_crs(4326).explor.." store_history=True silent=False shell_futures=True cell_id=6a232523-972c-469b-9ec9-a590d4412642> result=<folium.folium.Map object at 0x7f2551603210>>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [12]:
# Get IPF version for each of these
def get_ipf(sceneName):
    sat = sceneName[0]+sceneName[2]
    url = f'https://datapool.asf.alaska.edu/METADATA_SLC/{sat}/{sceneName}.iso.xml'
    print(url)
    r = requests.get(url)
    
    IPF = re.search(r'\(version(.*?)\)',r.text).group(1).strip()
    
    return IPF

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f255093ea90, execution_count=12 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f2550906310, raw_cell="# Get IPF version for each of these
def get_ipf(sc.." store_history=True silent=False shell_futures=True cell_id=2a0b3721-4c9b-43c9-b49c-feb4a2114a11> result=None>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [13]:
#subset['IPF'] = subset.sceneName.apply(get_ipf)

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f2550921350, execution_count=13 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f2551610e10, raw_cell="#subset['IPF'] = subset.sceneName.apply(get_ipf)" store_history=True silent=False shell_futures=True cell_id=048b03e7-57a3-435e-9bb2-c4893b155a4f> result=None>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [14]:
#subset.IPF

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f2550945750, execution_count=14 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f2550946250, raw_cell="#subset.IPF" store_history=True silent=False shell_futures=True cell_id=67d39b4d-6c32-4693-bf2e-158d0a26e2a9> result=None>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [15]:
# Only IPF>=3.4 has burstnumbers in the metadata, others require reverse-lookup based on TANX
def get_metadata_xml(session, params):
    root = hb.download_metadata(session, params)  
    
    return root

def get_ipf(root):
    ''' for consolidated XML metadata get version '''
    ipfnode = root.find('.//safe:software', {'safe':'http://www.esa.int/safe/sentinel-1.0'})
    
    return ipfnode.attrib['version']


def get_burstnumber(session, row, myburst, product_schema='./support/s1-level-1-product.xsd'):
    ''' given an ASF frame, determine relative burst number corresponding to standard esa burstid
    
    session = asf session from hyp3-isce2 
    row = geodataframe row of asf_search results
    myburst = geoseries for burst of interest
    
    Note: requires support/s1-level-1-product.xsd XML schema from SLC SAFE for parsing metadata
    '''
    params = hb.BurstParams(row.sceneName, myburst.subswath_name, row.polarization[:2], 1)
    
    # Get All XML metadata for SLC
    root = get_metadata_xml(session, params)
    #print(root)
    #IPF = get_ipf(root)
    #print(IPF)

    # Extract correct section of xml
    for product in root.findall('.//product'):
        prod_pol = product.find('polarisation').text
        prod_swath = product.find('swath').text
        
        if (prod_pol == params.polarization) and (prod_swath == params.swath):
            node = product.find('content')
            node.tag = 'product'
            string = lxml.etree.tostring(node, encoding='unicode')
            
    # Convert to python dictionary
    xs = xmlschema.XMLSchema(product_schema)
    parsed = xs.to_dict(string, validation='lax')[0]
    
    tanx = np.array([t['azimuthAnxTime'] for t in parsed['swathTiming']['burstList']['burst']])
    print(tanx)
    burstnum = np.argmin(np.abs(tanx - myburst.time_from_anx_sec))
    print(myburst.time_from_anx_sec)
    if np.abs(tanx[burstnum]-myburst.time_from_anx_sec)>1.5:
        burstnum=-1
    return burstnum

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f2551c53a50, execution_count=15 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f255092d150, raw_cell="# Only IPF>=3.4 has burstnumbers in the metadata, .." store_history=True silent=False shell_futures=True cell_id=5c2988d6-3b24-4d7c-aaac-5090f2639725> result=None>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [16]:
myburst # '137_IW2_292399'

,burst_id,subswath_name,relative_orbit_number,time_from_anx_sec,orbit_pass,geometry
2,365368,IW2,171,607.902997,ASCENDING,"MULTIPOLYGON Z (((119.91344 37.49934 0.00000, ..."


Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f25509452d0, execution_count=16 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f2551c3b590, raw_cell="myburst # '137_IW2_292399'" store_history=True silent=False shell_futures=True cell_id=b1f92345-7939-43f1-8515-3ae08e706b29> result=   burst_id subswath_name  relative_orbit_number  time_from_anx_sec  \
2    365368           IW2                    171         607.902997   

  orbit_pass                                           geometry  
2  ASCENDING  MULTIPOLYGON Z (((119.91344 37.49934 0.00000, ...  >,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [17]:
asf_session = hb.get_asf_session()
print(asf_session)

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f2551603710, execution_count=17 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f25c891a950, raw_cell="asf_session = hb.get_asf_session()
print(asf_sessi.." store_history=True silent=False shell_futures=True cell_id=1e91c851-2df4-47b1-93cc-c827d436d15d> result=None>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [18]:
# For small number of granules just iterate over pandas dataframe (slow but works)
import multiprocessing
#print(multiprocessing)

Bursts = []
'''
print(subset.iterrows())
for i,row in subset.iterrows():
    num = get_burstnumber(asf_session, row, myburst.iloc[0])
    Bursts.append(num)
    print(num)
'''
    
def search_burst(i):    
    for j,row in subset.iterrows():
        if j==i:
            break
    num = get_burstnumber(asf_session, row, myburst.iloc[0])
    text=str(i)+','+str(num)
    with open("burst_list.txt","a") as f:
        f.writelines(text)
        f.writelines("\n")
    
    
os.system("rm -rf burst_list.txt")    
pool = multiprocessing.Pool(8)
for i,row in subset.iterrows():
    hhhhhhh=0
print(i)
t=np.arange(i+1)
results = pool.map(search_burst, t)
pool.close()
pool.join()
print("end")
Burst=[]
Bursts=np.loadtxt("burst_list.txt",delimiter=",")
for i in range(len(Bursts)):
    for j in range(len(Bursts)):
        txt=Bursts[j]
        if int(txt[0])==i:
            Burst.append(int(txt[1]))
            break

444
[609.66790845 612.41618722 615.17474378 617.93124477 620.70007911
 623.44424677 626.20485888 628.96135988 631.72197199]
607.9029970747706
[582.092542   584.849043   587.60143288 590.35998944 593.12265711
 595.87710255 598.6356591  601.3921601  604.15071666 606.90721765]
607.9029970747706
[609.64660344 612.40516    615.16782766 617.92432866 620.67671855
 623.44349733 626.20616499 628.96061044 631.72122255]
607.9029970747706
[609.64671077 612.42582289 615.18437945 617.94088044 620.699437
 623.455938   626.21449455 628.97510666 631.73160766]
607.9029970747706[606.922915   609.68147155 612.44208366 615.19858466 617.95714122
 620.71158666 623.47014321 626.23075532 628.98931188]

607.9029970747706
[609.66947633 612.43008844 615.18864499 617.94514599 620.70164699
 623.46020355 626.22287121 628.98142777 631.73792877]
607.9029970747706
[609.67805444 612.43866655 615.19722311 617.95372411 620.70816955
 623.46878166 626.22939377 628.98589477 631.74650688]
607.9029970747706
[609.668807   612.4

TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [19]:
subset['burst'] = Burst

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f2551349cd0, execution_count=19 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f25c891a950, raw_cell="subset['burst'] = Burst" store_history=True silent=False shell_futures=True cell_id=be9e7f2c-483a-4b9e-bacb-bf635b2b3f07> result=None>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [20]:
subset['date'] = gpd.pd.to_datetime(subset.startTime).dt.strftime('%Y%m%d')

Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f25510eed10, execution_count=20 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f25509012d0, raw_cell="subset['date'] = gpd.pd.to_datetime(subset.startTi.." store_history=True silent=False shell_futures=True cell_id=b56370b9-660d-49b0-a2a7-9730c48e80ab> result=None>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [21]:
with gpd.pd.option_context('display.max_colwidth', None):
    display(subset.loc[:,['date','sceneName','burst']])
    print(subset.loc[:,['burst']])

,date,sceneName,burst
0,20240110,S1A_IW_SLC__1SDV_20240110T095716_20240110T095743_052043_064A1D_116A,-1
1,20240110,S1A_IW_SLC__1SDV_20240110T095650_20240110T095718_052043_064A1D_75E7,9
2,20231229,S1A_IW_SLC__1SDV_20231229T095651_20231229T095728_051868_064426_76AB,9
3,20231217,S1A_IW_SLC__1SDV_20231217T095651_20231217T095728_051693_063E16_4E22,9
4,20231205,S1A_IW_SLC__1SDV_20231205T095652_20231205T095729_051518_0637F8_E127,9
...,...,...,...
440,20170416,S1A_IW_SLC__1SDV_20170416T095634_20170416T095701_016168_01AB6F_383B,-1
441,20170416,S1A_IW_SLC__1SDV_20170416T095609_20170416T095636_016168_01AB6F_49EE,9
442,20170404,S1A_IW_SLC__1SDV_20170404T095634_20170404T095701_015993_01A612_82D6,-1
443,20170404,S1A_IW_SLC__1SDV_20170404T095608_20170404T095636_015993_01A612_CE0E,9


     burst
0       -1
1        9
2        9
3        9
4        9
..     ...
440     -1
441      9
442     -1
443      9
444     10

[445 rows x 1 columns]
Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f255114a090, execution_count=21 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f25c891a950, raw_cell="with gpd.pd.option_context('display.max_colwidth',.." store_history=True silent=False shell_futures=True cell_id=27359551-ff2f-48c3-9ec6-87e0c166a6bc> result=None>,),kwargs {}:


TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [22]:
# Get a summer scene from each year 
gf = gf.set_index(gpd.pd.to_datetime(gf.startTime))
gf.head()


,geometry,beamModeType,browse,bytes,centerLat,centerLon,faradayRotation,fileID,flightDirection,groupID,...,sceneName,sensor,startTime,stopTime,url,pgeVersion,fileName,frameNumber,burst,date
startTime,,,,,,,,,,,,,,,,,,,,,
2024-01-10 09:57:16.146000+00:00,"POLYGON ((118.55290 39.01779, 118.95319 37.397...",IW,None,7665833160,38.4153,120.1865,None,S1A_IW_SLC__1SDV_20240110T095716_20240110T0957...,ASCENDING,S1A_IWDV_0120_0127_052043_171,...,S1A_IW_SLC__1SDV_20240110T095716_20240110T0957...,C-SAR,2024-01-10T09:57:16.146Z,2024-01-10T09:57:43.102Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.71,S1A_IW_SLC__1SDV_20240110T095716_20240110T0957...,121,-1,20240110
2024-01-10 09:56:50.369000+00:00,"POLYGON ((118.92621 37.52652, 119.33179 35.847...",IW,None,7971013535,36.8947,120.5317,None,S1A_IW_SLC__1SDV_20240110T095650_20240110T0957...,ASCENDING,S1A_IWDV_0115_0122_052043_171,...,S1A_IW_SLC__1SDV_20240110T095650_20240110T0957...,C-SAR,2024-01-10T09:56:50.369Z,2024-01-10T09:57:18.284Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.71,S1A_IW_SLC__1SDV_20240110T095650_20240110T0957...,116,9,20240110
2023-12-29 09:56:51.007000+00:00,"POLYGON ((118.78477 38.08155, 119.32674 35.847...",IW,None,10705033744,37.1721,120.4627,None,S1A_IW_SLC__1SDV_20231229T095651_20231229T0957...,ASCENDING,S1A_IWDV_0115_0123_051868_171,...,S1A_IW_SLC__1SDV_20231229T095651_20231229T0957...,C-SAR,2023-12-29T09:56:51.007Z,2023-12-29T09:57:28.153Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.71,S1A_IW_SLC__1SDV_20231229T095651_20231229T0957...,116,9,20231229
2023-12-17 09:56:51.813000+00:00,"POLYGON ((118.78420 38.08128, 119.32612 35.847...",IW,None,10704952926,37.1720,120.4619,None,S1A_IW_SLC__1SDV_20231217T095651_20231217T0957...,ASCENDING,S1A_IWDV_0115_0123_051693_171,...,S1A_IW_SLC__1SDV_20231217T095651_20231217T0957...,C-SAR,2023-12-17T09:56:51.813Z,2023-12-17T09:57:28.952Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.71,S1A_IW_SLC__1SDV_20231217T095651_20231217T0957...,116,9,20231217
2023-12-05 09:56:52.420000+00:00,"POLYGON ((118.78315 38.08145, 119.32542 35.846...",IW,None,10705000738,37.1716,120.4612,None,S1A_IW_SLC__1SDV_20231205T095652_20231205T0957...,ASCENDING,S1A_IWDV_0115_0123_051518_171,...,S1A_IW_SLC__1SDV_20231205T095652_20231205T0957...,C-SAR,2023-12-05T09:56:52.420Z,2023-12-05T09:57:29.580Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.71,S1A_IW_SLC__1SDV_20231205T095652_20231205T0957...,116,9,20231205


Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f25511481d0, execution_count=22 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f25509012d0, raw_cell="# Get a summer scene from each year 
gf = gf.set_i.." store_history=True silent=False shell_futures=True cell_id=b74ac77a-3f48-405d-b7f7-c6a8000f573b> result=                                                                           geometry  \
startTime                                                                             
2024-01-10 09:57:16.146000+00:00  POLYGON ((118.55290 39.01779, 118.95319 37.397...   
2024-01-10 09:56:50.369000+00:00  POLYGON ((118.92621 37.52652, 119.33179 35.847...   
2023-12-29 09:56:51.007000+00:00  POLYGON ((118.78477 38.08155, 119.32674 35.847...   
2023-12-17 09:56:51.813000+00:00  POLYGON ((118.78420 38.08128, 119.32612 35.847...   
2023-12-05 09:56:52.420000+0

TypeError: DfMagics.run() takes 1 positional argument but 2 were given

In [23]:
with open(f'../stack.txt', 'w') as f:
    f.write('')
for i in range(len(subset)):
    cmd = str(subset.iloc[i].sceneName)+','+str(subswath)+','+str(subset.iloc[i].burst)+'\n'
    if str(subset.iloc[i].burst) !='-1':
        with open(f'../p/data/stack.txt', 'a') as f:
            f.write(cmd)
    else:
        print('跳过')

跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
跳过
Error in callback <bound method DfMagics.run of <__main__.DfMagics object at 0x7f25c88fd610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f2551167f10, execution_count=23 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f255115e4d0, raw_cell="with open(f'../stack.txt', 'w') as f:
    f.write(.." store_history=True silent=False shell_futures=True cell_id=bc97be3f-b88f-440a-80e9-8570050ef6f9> result=None>,),kwargs {

TypeError: DfMagics.run() takes 1 positional argument but 2 were given